# Data Preparation - Robots.txt WARC Captures of Top-K Websites

Objective: extract the robots.txt WARC records of top-k ranked websites from [Common Crawl's robots.txt dataset](https://commoncrawl.org/2016/09/robotstxt-and-404-redirect-data-sets/).


## Download Robots.txt Records of Top-K Websites

Note: the general procedure how to extract records for a large list of domains is described in this [notebook](https://github.com/commoncrawl/cc-notebooks/blob/main/cc-index-table/bulk-url-lookups-by-table-joins.ipynb#Variants-and-Further-Optimizations) in more detail.

## Selection of Top-K Websites

See the README in [data/top-k-sites](../../data/top-k-sites/README.md) for how the list is compiled from Tranco lists.

We now convert the list into [Parquet](https://parquet.apache.org/) file format...

In [1]:
import pandas as pd

df = pd.read_csv('../../data/top-k-sites/tranco/tranco_combined.txt.gz', sep='\t', names=['rank', 'host'])
df.head()

,rank,host
0,1,google.com
1,2,facebook.com
2,3,microsoft.com
3,4,youtube.com
4,5,akamaiedge.net


In [2]:
# rows and columns of the list
df.shape

(2042066, 2)

In [3]:
# add the registered (aka. pay-level) domain to top-k list
import tldextract

df['domain'] = df['host'].apply(lambda host: tldextract.extract(host).registered_domain)

In [4]:
df[df['domain'] != df['host']].head(10)

,rank,host,domain
10,11,www.google.com,google.com
15,16,data.microsoft.com,microsoft.com
25,26,ctldl.windowsupdate.com,windowsupdate.com
26,27,events.data.microsoft.com,microsoft.com
31,32,ftl.netflix.com,netflix.com
35,36,cloud.netflix.com,netflix.com
37,38,play.google.com,google.com
38,39,en.wikipedia.org,wikipedia.org
40,41,safebrowsing.googleapis.com,googleapis.com
42,43,prod.cloud.netflix.com,netflix.com


In [5]:
# save as Parquet
df.to_parquet('../../data/top-k-sites/tranco/tranco_combined.zstd.parquet', compression='zstd', index=False)

## Bulk Lookup by Table Join

The lookup of all top-k websites in the [columnar index](https://commoncrawl.org/2018/03/index-to-warc-files-and-urls-in-columnar-format/) requires to

- upload the Parquet domain list to S3 (`mybucket` is a placeholder for a bucket in `us-east-1`)
  ```bash
  aws s3 cp ../../data/top-k-sites/tranco/tranco_combined.zstd.parquet s3://mybucket/robotstxt-experiments/domain-top-k/
  ```
- register the domain list as table in [Amazon Athena](https://aws.amazon.com/athena/)
  - navigate to the [Athena query editor](https://console.aws.amazon.com/athena/home?region=us-east-1#/query-editor) and
  - create a database "robotsexperiments" by executing the following statement:
    ```sql
    CREATE DATABASE robotsexperiments;
    ```
  - register the table "topdomains":
    ```sql
    CREATE EXTERNAL TABLE IF NOT EXISTS robotsexperiments.topdomains (
      `rank`   int,
      `host`   string,
      `domain` string
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
    WITH SERDEPROPERTIES (
      'serialization.format' = '1'
    ) LOCATION 's3://mybucket/robotstxt-experiments/domain-top-k/'
    TBLPROPERTIES ('has_encrypted_data'='false');
    ```
  - and verify whether the table is imported properly and contains the expected number of rows
    ```sql
    SELECT * FROM robotsexperiments.topdomains limit 10;
  
    SELECT COUNT(*) FROM robotsexperiments.topdomains;
    ```

Finally, the bulk lookup is done by a table join with the [Common Crawl's columnar index](ttps://commoncrawl.org/2018/03/index-to-warc-files-and-urls-in-columnar-format/), we
- select only the most recent record per same robots.txt URL (the crawler might fetch the robots.txt repeatedly during a monthly crawling running over almost two weeks)
- extract WARC record locations for later processing of robots.txt records
- MIME types (HTTP Content-Type header and identified by content)
- fetch time and status
- and redirect locations (since CC-MAIN-2019-47) in order to "follow" redirects

```sql
WITH allrobots AS (
  SELECT topdomains.host as host,
         topdomains.domain as domain,
         topdomains.rank as rank,
         cc.url_host_tld,
         cc.url_host_registered_domain,
         cc.url_host_name,
         cc.url,
         cc.url_path,
         cc.url_query,
         cc.fetch_time,
         cc.fetch_status,
         cc.warc_filename,
         cc.warc_record_offset,
         cc.warc_record_length,
         cc.fetch_redirect,
         cc.content_mime_type,
         cc.content_mime_detected,
         -- enumerate records of same URL, most recent first
         ROW_NUMBER() OVER(PARTITION BY cc.url ORDER BY cc.fetch_time DESC) AS n
  FROM "ccindex"."ccindex" AS cc
    RIGHT OUTER JOIN "robotsexperiments"."topdomains" AS topdomains
    ON topdomains.host = cc.url_host_name
  WHERE cc.crawl = 'CC-MAIN-2022-33'
    AND cc.subset = 'robotstxt'
    AND cc.url_path = '/robots.txt'
    AND cc.url_query IS NULL
SELECT *
 FROM allrobots
-- select only the first (most recent) record of the same URL
WHERE allrobots.n = 1;
```

The query extracts the robots.txt records for a single monthly crawl (CC-MAIN-2022-33). We repeat this
- for all crawls since August 2016, run in August or February (one data point every six months)
- for the years 2020 - 2025 we include all crawls to get the maximum coverage.

Based on this selection, we need to run the query on 48 crawls from August 2016 until February 2025. See [top-k-sample/crawls.txt](../../data/top-k-sample/crawls.txt) for the list of crawl identifiers.

Running the queries is done by a Python script [get-robotstxt-captures-athena.py](../script/get-robotstxt-captures-athena.py) based on [PyAthena](https://pypi.org/project/PyAthena/). The results are stored on S3 in Parquet format.

### Result Verification

We register the results for all crawls as an Athena table:
```sql
CREATE EXTERNAL TABLE IF NOT EXISTS `robotsexperiments`.`domain_top_k_sample` (
  `host` string,
  `domain` string,
  `rank` int,
  `url_host_tld` string,
  `url_host_registered_domain` string,
  `url_host_name` string,
  `url` string,
  `url_path` string,
  `url_query` string,
  `fetch_time` timestamp,
  `fetch_status` int,
  `warc_filename` string,
  `warc_record_offset` int,
  `warc_record_length` int,
  `fetch_redirect` string,
  `content_mime_type` string,
  `content_mime_detected` string
)
PARTITIONED BY (`crawl` string)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
LOCATION 's3://mybucket/robotstxt-experiments/domain-top-k-sample/'
TBLPROPERTIES ('classification' = 'parquet');
```

After loading the partitions we can count the number of robots.txt captures found per crawl:
```sql
select
  count(*) as n_captures,
  crawl
from domain_top_k_sample
group by crawl order by crawl;
```

In [6]:
import pandas as pd

robotstxt_captures_found = pd.read_csv('../../data/top-k-sample/robotstxt-captures-counts.csv')
robotstxt_captures_found

,n_captures,crawl
0,604440,CC-MAIN-2016-36
1,717829,CC-MAIN-2017-09
2,615175,CC-MAIN-2017-34
3,807732,CC-MAIN-2018-09
4,980026,CC-MAIN-2018-34
5,1033590,CC-MAIN-2019-09
6,864256,CC-MAIN-2019-35
7,779791,CC-MAIN-2020-05
8,838228,CC-MAIN-2020-10
9,815695,CC-MAIN-2020-16


The top-k sites where selected in 2020 - 2024. The number of robots.txt captures is close to 1 million for these years, which means a coverage of 50%. For 2016 and 2017 the coverage is lower. This is not unexpected.

There are several reasons why there is no robots.txt capture for a site (host name):
- the crawler didn't visit the site in this month. Consequently, there is also no robots.txt capture.
- the robots.txt fetch failed before a HTTP connection could be established. This usually means that also no content was crawled at all from the given site.

In addition, the robots.txt capture can be
- a HTTP 404 " the site has no robots.txt
- any other non-success HTTP code
- including redirects - which we need to follow, see below

The crawler follows redirects as specified per [RFC 9309](https://www.rfc-editor.org/rfc/rfc9309.html#name-redirects). However, the redirect target is only archived if the following conditions are given:
- it's a text file (not HTML or anything else)
- the URL path is "/robots.txt" and URL query is empty, if not then the redirect location is required to be
  - allowed by the robots.txt (maybe the robots.txt of a different site)
  - not excluded per URL filter

The strict robots.txt archiving policies are necessary to avoid that an attacker is able to put secret or sensitive content into the robots.txt archives by pointing a robots.txt redirect to a path on a different site.